In [1]:
%pip install pandas==1.5.1
%pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.2 MB 20.4 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 27.9 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from torchvision.transforms.functional import hflip
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from matplotlib import image as mpimg
import re
from PIL import Image
import pandas as pd
import datetime
from torchmetrics import TweedieDevianceScore

import math
from torch import default_generator, randperm
from torch._utils import _accumulate
from torch.utils.data.dataset import Subset

from sklearn.metrics import fbeta_score, make_scorer, mean_poisson_deviance
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import fbeta_score, make_scorer



In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

PATH='/content/drive/MyDrive/MLproject/'

with open(PATH+"listings.pkl", "rb") as fh:
    data = pickle.load(fh)
with open(PATH+"test_listings_image.pkl", "rb") as fh:
    data_test = pickle.load(fh)

cuda


In [17]:
data.columns

Index(['Are Pets Allowed', 'Day of Advertisement Created',
       'Day of Date Available From', 'Demand', 'Floor', 'Geo Canton',
       'Has Balcony', 'Has Cabletv', 'Has Elevator', 'Has Fireplace',
       'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Listing Description', 'Listing Title',
       'Livingspace', 'Number Of Rooms', 'Number Of Rooms Cleaned',
       'Number of Documents', 'Number of Images', 'Price Extra Normalized',
       'Price Gross Normalized', 'Price M2 Normalized', 'Price Net Normalized',
       'Property Reference Id', 'Size M2 Normalized', 'Subcategory En Idx',
       'Year Built', 'Year Lastrenovated'],
      dtype='object')

In [6]:
data.sample(5)


,Are Pets Allowed,Category Idx,Day of Advertisement Created,Day of Date Available From,Demand,Floor,Geo Canton,Geo City,Geo Zip,Has Balcony,...,Number of Images,Price Extra Normalized,Price Gross Normalized,Price M2 Normalized,Price Net Normalized,Property Reference Id,Size M2 Normalized,Subcategory En Idx,Year Built,Year Lastrenovated
18307,False,APPT,"January 25, 2017","April 1, 2017",0.0,2.0,AG,Anglikon,5611,False,...,4,280.0,2090.0,18.0,1810.0,1367.22.5664,116.0,Apartment,NaN,NaN
13405,True,APPT,"August 30, 2017",NaN,1.0,1.0,SO,Zuchwil,4528,False,...,10,210.0,1100.0,20.0,890.0,4612.50.8394,55.0,Apartment,NaN,NaN
10722,True,APPT,"April 13, 2022","May 16, 2022",0.0,0.0,VD,Ecublens VD,1024,NaN,...,2,255.0,2075.0,26.0,1820.0,9880.26.7822,80.0,Apartment,2018.0,2018.0
15890,NaN,APPT,"April 16, 2021","July 1, 2021",55.0,5.0,ZH,Zürich,8041,NaN,...,4,245.0,2800.0,33.0,2555.0,6331.50.3690,85.0,Apartment,2017.0,2017.0
11642,NaN,APPT,"May 15, 2019","June 1, 2019",7.0,3.0,GE,Genève,1207,NaN,...,7,370.0,4870.0,37.0,4500.0,3721.61.6571,133.0,Apartment,NaN,NaN


In [7]:
features = data[['Are Pets Allowed', 'Category Idx', 'Day of Advertisement Created',
       'Day of Date Available From', 'Floor', 'Geo Canton',
       'Geo City', 'Geo Zip', 'Has Balcony', 'Has Cabletv', 'Has Elevator',
       'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Listing Description', 'Listing Title',
       'Livingspace', 'Number Of Rooms', 'Number Of Rooms Cleaned',
       'Number of Documents', 'Number of Images', 'Price Extra Normalized',
       'Price Gross Normalized', 'Price M2 Normalized', 'Price Net Normalized',
       'Property Reference Id', 'Size M2 Normalized', 'Subcategory En Idx',
       'Year Built', 'Year Lastrenovated']]

label = data['Demand']


In [19]:
def accuracy(y_pred,y,tol=0):
    numpy_y=y.to_numpy()
    rounded_y_pred=y_pred.round()
    assert len(rounded_y_pred)==len(numpy_y)
    return (np.abs(rounded_y_pred-numpy_y)<=tol).sum()/len(rounded_y_pred)


def evaluate_model(y_true,y_pred):
    prediction=y_pred.copy()
    prediction[prediction<=0]=10**(-10)
    return mean_poisson_deviance(y_true,prediction)
    

scoring=make_scorer(evaluate_model,greater_is_better=False)




In [26]:
seed=np.random.randint(100)
DICT_GEO_CANTON={'ZH':0, 'VD':1, 'BE':2, 'AG':3, 'FR':4, 'SG':5, 'LU':6, 'SO':7, 'SH':8, 'GE':9, 'VS':10,
       'SZ':11, 'BS':12, 'ZG':13, 'BL':14, 'NE':15, 'TI':16, 'GR':17, 'TG':18, 'AR':19, 'UR':20, 'GL':21,
       'NW':22, 'JU':22}

DICT_SUBCATEGORY={'Apartment':0, 'Duplex':1, 'Row house':2, 'Roof flat':3, 'Attic flat':4,
       'Single house':5, 'Single Room':6, 'Studio':7, 'Furnished dwelling':8,
       'Loft':9, 'Mansard':10, 'Farm house':11}


#identifying the boolean variables
bool_features=['Are Pets Allowed', 'Has Balcony', 'Has Cabletv',
               'Has Elevator', 'Has Fireplace', 'Has Garage',
               'Has Parking', 'Is New Construction', 'Is New Construction Potential', 
               'Is Tenant2Tenant', 'Is Wheelchairaccessible']
print(bool_features)
data_bool=data[bool_features]


#The ordinal features are:
ordinal_features=['Floor','Livingspace', 'Number Of Rooms',
       'Number Of Rooms Cleaned', 'Number of Documents', 'Number of Images',
       'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 
       'Size M2 Normalized', 'Year Built',
       'Year Lastrenovated']

categorical_features={'Geo Canton':DICT_GEO_CANTON,'Subcategory En Idx':DICT_SUBCATEGORY}

#Which leaves us with 'Property Reference Id ',Listing Description' and 'Listing Title' which we will not use for regression purpose
data=data.drop(['Property Reference Id','Listing Description','Listing Title'],axis=1)

null_count=pd.concat([data.isnull().sum().rename('NaN train set'),data_test.isnull().sum().rename('NaN test set')],axis=1)

iter_imputer = IterativeImputer(estimator=ExtraTreesRegressor(n_estimators=10, random_state=0),random_state=42,max_iter=20)

temp=data.copy()

iter_imputed = iter_imputer.fit_transform(data[ordinal_features])

temp.loc[:,ordinal_features]=iter_imputed




['Are Pets Allowed', 'Has Balcony', 'Has Cabletv', 'Has Elevator', 'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction', 'Is New Construction Potential', 'Is Tenant2Tenant', 'Is Wheelchairaccessible']


/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
<ipython-input-26-cca028cf8e98>:41: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  temp.loc[:,ordinal_features]=iter_imputed


In [31]:

#Deal with 'Day of Advertisement Created' and 'Day of Date Available From'
def compute_diff_date(X: pd.DataFrame, y=None):
    X_copy=X.copy()
    threshold=datetime.datetime(2022,9,1)
    X_copy['Days Between September and Created']=pd.to_datetime(X_copy['Day of Advertisement Created']).apply(lambda x:(threshold-x).days)
    X_copy['Days Between September and Available']=pd.to_datetime(X_copy['Day of Date Available From']).apply(lambda x:(threshold-x).days)                                                           
    X_copy.drop(['Day of Advertisement Created','Day of Date Available From'],axis=1,inplace=True)
    return X_copy

diff_date_transformer = FunctionTransformer(compute_diff_date)

print('Dealing with date')
display(diff_date_transformer.transform(data))

Dealing with date


,Are Pets Allowed,Category Idx,Demand,Floor,Geo Canton,Geo City,Geo Zip,Has Balcony,Has Cabletv,Has Elevator,...,Price Extra Normalized,Price Gross Normalized,Price M2 Normalized,Price Net Normalized,Size M2 Normalized,Subcategory En Idx,Year Built,Year Lastrenovated,Days Between September and Created,Days Between September and Available
0,NaN,APPT,25.0,0.0,ZH,Dübendorf,8600,NaN,NaN,NaN,...,550.0,4460.0,19.0,3910.0,232.0,Apartment,2016.0,2016.0,602,549.0
1,NaN,APPT,2.0,1.0,VD,Le Mont-sur-Lausanne,1052,NaN,NaN,NaN,...,120.0,1345.0,30.0,1225.0,45.0,Apartment,NaN,NaN,1018,914.0
2,False,APPT,0.0,1.0,BE,Langenthal,4900,False,False,False,...,250.0,1570.0,15.0,1320.0,103.0,Apartment,NaN,NaN,1996,NaN
3,True,APPT,4.0,1.0,AG,Fislisbach,5442,NaN,True,NaN,...,270.0,1480.0,17.0,1210.0,86.0,Apartment,1985.0,1985.0,421,488.0
4,False,APPT,3.0,0.0,FR,Bulle,1630,False,False,False,...,300.0,1490.0,19.0,1190.0,79.0,Apartment,NaN,NaN,1763,1857.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,False,APPT,2.0,2.0,SG,St. Gallen,9000,False,False,False,...,170.0,1350.0,15.0,1180.0,90.0,Apartment,NaN,NaN,1701,1614.0
19996,True,APPT,1.0,2.0,VD,Vevey,1800,NaN,True,True,...,190.0,1940.0,23.0,1750.0,84.0,Apartment,1968.0,1968.0,302,212.0
19997,NaN,APPT,2.0,5.0,GE,Grand-Lancy,1212,NaN,NaN,NaN,...,275.0,2175.0,32.0,1900.0,68.0,Apartment,NaN,NaN,321,442.0
19998,NaN,APPT,2.0,7.0,VD,Nyon,1260,NaN,NaN,NaN,...,240.0,3540.0,37.0,3300.0,95.0,Apartment,NaN,NaN,700,488.0


In [32]:
other_categorical_features=['Category Idx','Geo City','Geo Zip']
data=data.drop(other_categorical_features,axis=1)

In [35]:
categorical_features={'Geo Canton':DICT_GEO_CANTON,'Subcategory En Idx':DICT_SUBCATEGORY}


In [36]:
def fill_nan(X: pd.DataFrame,bool_features,categorical_features,ordinal_features,bool_fill=-99999, y=None):
    df=X.copy() #We don't want to modify the dataset
    
    #Dealing with categorical values
    for bool_feature in bool_features:
        df.loc[:,bool_feature]=df.loc[:,bool_feature].fillna(bool_fill)
        df.loc[df[bool_feature] == False, bool_feature] = 0
        df.loc[data[bool_feature] == True, bool_feature] = 1
    
    #Dealing with ordinal values
    for ordinal_feature in ordinal_features:
        df[ordinal_feature].fillna(df[ordinal_feature].mean(),inplace=True)
        
    #Dealing with categorical features
    for categorical_feature in categorical_features.keys():
        dict_replace=categorical_features[categorical_feature]
        df[categorical_feature]=df[categorical_feature].replace(dict_replace)
        df[categorical_feature].fillna(len(dict_replace)+1,inplace=True)
        #df[categorical_feature].fillna(bool_fill,inplace=True)
    return df

na_fill_estimator = FunctionTransformer(fill_nan,kw_args={'bool_features':bool_features,
                                                          'categorical_features':categorical_features,
                                'ordinal_features':ordinal_features+['Days Between September and Created',
                                                                     'Days Between September and Available']})


features_to_encode=bool_features
col_trans = make_column_transformer(
                        (OneHotEncoder(),features_to_encode),
                        remainder = "passthrough"
                        )
print('One Hot Encoded data:')
print(col_trans.fit_transform(na_fill_estimator.transform(diff_date_transformer.transform(data))))
print(col_trans.fit_transform(na_fill_estimator.transform(diff_date_transformer.transform(data))).shape)



One Hot Encoded data:
[[1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 2.0160000e+03
  6.0200000e+02 5.4900000e+02]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.9917911e+03
  1.0180000e+03 9.1400000e+02]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 1.9917911e+03
  1.9960000e+03 1.0280996e+03]
 ...
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.9917911e+03
  3.2100000e+02 4.4200000e+02]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.9917911e+03
  7.0000000e+02 4.8800000e+02]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 ... 1.9917911e+03
  1.9390000e+03 1.8260000e+03]]
(20000, 49)


In [39]:
binary_features = ['Are Pets Allowed', 'Has Balcony', 'Has Cabletv', 'Has Elevator',
                  'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
                  'Is New Construction Potential', 'Is Tenant2Tenant',
                  'Is Wheelchairaccessible']
for column in binary_features:
  data.loc[data[column] == False, column] = 0
  data.loc[data[column] == True, column] = 1

data = pd.get_dummies(data, prefix=['Geo'], columns=['Geo Canton'])

data['Year Lastrenovated'].fillna(0, inplace = True)


In [40]:
output_cols = list(data['Demand'])

# input_cols = data[['Are Pets Allowed', 'Floor',
#        'Has Balcony', 'Has Cabletv', 'Has Elevator',
#        'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
#        'Is New Construction Potential', 'Is Tenant2Tenant',
#        'Is Wheelchairaccessible', 'Livingspace', 'Number Of Rooms', 'Number Of Rooms Cleaned',
#        'Number of Documents', 'Number of Images', 'Price Extra Normalized',
#        'Price Gross Normalized', 'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized', 
#        'Year Lastrenovated', 
#        'Geo Canton']]

input_cols = data[['Are Pets Allowed', 'Floor',
       'Has Balcony', 'Has Cabletv', 'Has Elevator',
       'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Livingspace', 'Number Of Rooms', 'Number Of Rooms Cleaned',
       'Number of Documents', 'Number of Images', 'Price Extra Normalized',
       'Price Gross Normalized', 'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized', 
       'Year Lastrenovated', 
       'Geo_AG', 'Geo_AR', 'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 
       'Geo_GL', 'Geo_GR', 'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 
       'Geo_SH', 'Geo_SO', 'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG', 'Geo_ZH']]

# Normalization of feature columns

def mean_norm(df_input):
    return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)

input_cols_norm = mean_norm(input_cols)
input_cols_norm.fillna(input_cols_norm.median(), inplace=True)
input_cols_norm.sample(10)

,Are Pets Allowed,Floor,Has Balcony,Has Cabletv,Has Elevator,Has Fireplace,Has Garage,Has Parking,Is New Construction,Is New Construction Potential,...,Geo_SH,Geo_SO,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH
3416,-0.832208,-0.602958,-0.200172,-1.317803,0.798902,-0.156072,-1.111143,-0.916588,-0.079381,-0.125677,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123
2568,-0.832208,-1.067466,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,-0.079381,-0.125677,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488
11712,-0.832208,-0.602958,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,-0.079381,-0.125677,...,-0.099737,-0.196833,-0.097148,7.558881,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
16439,-0.832208,-0.138449,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,-0.079381,-0.125677,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
1128,-0.832208,-0.602958,-0.200172,-1.317803,-1.251618,-0.156072,0.899897,1.090902,-0.079381,-0.125677,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
11339,-0.832208,0.326060,-0.200172,-1.317803,0.798902,-0.156072,0.899897,1.090902,-0.079381,-0.125677,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123
3243,-0.832208,-0.138449,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,-0.079381,-0.125677,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123
5562,-0.832208,-0.138449,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,-0.079381,-0.125677,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
5324,1.201510,-1.067466,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,-0.079381,-0.125677,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123
10516,-0.832208,0.326060,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,-0.079381,-0.125677,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123


In [42]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.autograd import Variable
import torchvision.models as models

In [43]:
inputs_array, targets_array1 = np.array(input_cols_norm.to_numpy()), label.to_numpy()
targets_array=[]
#We add brakets for tensor structure... (can be done differently)
for elt in targets_array1:
    targets_array.append([elt])
targets_array = np.array(targets_array)

print(type(inputs_array))
print(type(targets_array))
print(inputs_array)
print(targets_array)

inputs = torch.from_numpy(inputs_array).to(torch.float32)
targets = torch.from_numpy(targets_array).to(torch.float32)
dataset = TensorDataset(inputs, targets)

print(dataset)


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[-0.83220833 -1.06746644 -0.20017178 ... -0.10497413 -0.10473044
   1.3821235 ]
 [-0.83220833 -0.60295764 -0.20017178 ... -0.10497413 -0.10473044
  -0.72348817]
 [-0.83220833 -0.60295764 -0.20017178 ... -0.10497413 -0.10473044
  -0.72348817]
 ...
 [-0.83220833  1.25507753 -0.20017178 ... -0.10497413 -0.10473044
  -0.72348817]
 [-0.83220833  2.18409511 -0.20017178 ... -0.10497413 -0.10473044
  -0.72348817]
 [-0.83220833 -1.06746644 -0.20017178 ... -0.10497413 -0.10473044
  -0.72348817]]
[[25.]
 [ 2.]
 [ 0.]
 ...
 [ 2.]
 [ 2.]
 [ 3.]]


In [44]:
VAL_PERCENT = 0.2
NUM_ROWS = int(len(data))
val_size = int(NUM_ROWS * VAL_PERCENT)
train_size = NUM_ROWS - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

print(len(train_ds), len(val_ds))

16000 4000


In [45]:
BATCH_SIZE = 128

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

for xb, yb, in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[-0.8322,  0.3261, -0.2002,  ..., -0.1050, -0.1047,  1.3821],
        [-0.8322,  0.7906, -0.2002,  ..., -0.1050, -0.1047, -0.7235],
        [-0.8322, -1.0675, -0.2002,  ..., -0.1050, -0.1047, -0.7235],
        ...,
        [-0.8322, -0.1384, -0.2002,  ..., -0.1050, -0.1047,  1.3821],
        [-0.8322,  0.3261, -0.2002,  ..., -0.1050, -0.1047, -0.7235],
        [ 1.2015, -0.6030, -0.2002,  ..., -0.1050, -0.1047,  1.3821]])
targets: tensor([[  9.],
        [  7.],
        [  3.],
        [  0.],
        [  5.],
        [  0.],
        [  2.],
        [ 10.],
        [  8.],
        [  1.],
        [  3.],
        [ 16.],
        [  2.],
        [  1.],
        [  0.],
        [  3.],
        [ 10.],
        [ 18.],
        [  0.],
        [  1.],
        [  3.],
        [  2.],
        [  0.],
        [  2.],
        [  1.],
        [  4.],
        [  6.],
        [  0.],
        [ 12.],
        [  2.],
        [ 12.],
        [  4.],
        [  0.],
        [  4.],
     

In [147]:
#Creation of linear regression model

in_cols = input_cols.columns
out_cols = ['Demand']

input_size = len(in_cols)
output_size = len(out_cols)

class PredictModel(nn.Module):
    def __init__(self):
        # super().__init__()
        # self.linear = nn.Linear(input_size, output_size)
        super(PredictModel, self).__init__()
        self.layer_1 = nn.Linear(input_size, 16)
        self.layer_2 = nn.Linear(16, 32)
        # self.layer_3 = nn.Linear(32, 32)
        self.layer_3 = nn.Linear(32, 16)
        self.layer_out = nn.Linear(16, output_size)
        
        self.relu = nn.ReLU()
    
    def forward(self, xb):
        # out = self.linear(xb)
        x = self.relu(self.layer_1(xb))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        # x = self.relu(self.layer_4(x))
        x = self.layer_out(x)
        return x

    def training_step(self, batch):
        inputs, targets = batch
        out = self(inputs)
        loss = F.l1_loss(out, targets)
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        out = self(inputs)
        loss = F.l1_loss(out, targets)
        return {'val_loss': loss.detach()}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        if (epoch+1) % 20 == 0 or epoch == num_epochs - 1:
            print("Epoch[{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [148]:
#Defining our model 
model = PredictModel()
list(model.parameters())

#Training loop
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history=[]
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

result = evaluate(model, val_loader)
print(result)




{'val_loss': 8.672859191894531}


In [149]:
epochs = 300
lr = 0.001
history = fit(epochs, lr, model, train_loader, val_loader)

Epoch[20], val_loss: 7.6603
Epoch[40], val_loss: 6.6979
Epoch[60], val_loss: 6.1633
Epoch[80], val_loss: 6.0777
Epoch[100], val_loss: 6.0180
Epoch[120], val_loss: 5.9679
Epoch[140], val_loss: 5.9253
Epoch[160], val_loss: 5.8892
Epoch[180], val_loss: 5.8487
Epoch[200], val_loss: 5.8148
Epoch[220], val_loss: 5.7898
Epoch[240], val_loss: 5.7651
Epoch[260], val_loss: 5.7445
Epoch[280], val_loss: 5.7309
Epoch[300], val_loss: 5.7153


In [172]:
# We try to test our model on a random value of the test set

def predict_single(input, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)         
    prediction = predictions[0].detach()
    return prediction

input, target = val_ds[50]
prediction = predict_single(input, model)

print("Input:", input)
print("Target:", target)
print("Prediction:", prediction)
print()


# Return the entire predictions (Tensor Y_label)

def predict_all(val_ds, model):
    predictions = []
    targets = []
    for i in range(len(val_ds)):
       input, target = val_ds[i]
       prediction = predict_single(input, model)
       predictions.append(prediction.numpy()[0])
       targets.append(target)
    return predictions, targets

predictions, targets = predict_all(val_ds, model)

#On arrondi à l'entier le plus proche pour effectuer notre prédiction et on remplace les valeurs négatives
# predictions = np.round(predictions)
predictions = [max(10**(-0.0000005),k) for k in predictions]

# print(predictions)
# print(targets)


Input: tensor([ 1.2015e+00, -6.0296e-01, -2.0017e-01,  7.5878e-01,  7.9890e-01,
        -1.5607e-01,  8.9990e-01, -9.1659e-01, -7.9381e-02, -1.2568e-01,
        -1.0904e-01, -5.3497e-02,  1.8278e-01,  1.6239e-01,  1.6239e-01,
        -1.4841e+00, -3.8068e-01, -5.2332e-01, -5.7156e-02, -9.6593e-02,
         1.1888e-03,  1.8267e-01, -9.0298e-01, -3.5576e-01, -4.9047e-02,
         3.9114e+00, -2.3704e-01, -1.9058e-01, -1.5996e-01, -2.1080e-01,
        -4.3629e-02, -7.2297e-02, -1.2248e-02, -2.2170e-01, -1.3672e-01,
        -4.6417e-02, -2.6883e-01, -9.9737e-02, -1.9683e-01, -9.7148e-02,
        -1.3229e-01, -1.4770e-01, -5.1055e-02, -2.5962e-01, -1.0497e-01,
        -1.0473e-01, -7.2349e-01])
Target: tensor([0.])
Prediction: tensor([1.2987])



In [173]:
#Computation of the accurcy of our model on test set
EPSILON = 2

def accuracy(val_ds, model, EPSILON):
    corrects = 0
    for i in range(len(val_ds)):
        input, target = val_ds[i]
        prediction = predict_single(input, model)
        prediction = np.round(prediction)
        if prediction <= target + EPSILON and prediction >= target - EPSILON:
            corrects+=1
    accuracy = corrects/len(val_ds)
    print("Accuracy for EPSILON={}:".format(EPSILON), accuracy)
    return accuracy

accuracy(val_ds, model, EPSILON)



#Using Mean Poisson deviance
from sklearn.metrics import mean_poisson_deviance

print("Mean Poisson deviance:", mean_poisson_deviance(targets, predictions))




Accuracy for EPSILON=2: 0.545
Mean Poisson deviance: 8.305937705205329


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:746: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = np.asarray(array, order=order, dtype=dtype)
